In [2]:
import ujson
import pandas as pd
from Scripts.Helper_Functions import TOOI_organisaties

In [3]:
with open(f"SFTP_provincies.json", "r") as file:
    beschikkingen = ujson.load(file)

organisaties = TOOI_organisaties(orgs=['provincies'])

In [4]:
year_counts = {}
for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    if org_name not in year_counts:
        year_counts[org_name]={}
    year_counts[org_name][k_split[-1]] = len(v)

df = pd.DataFrame(year_counts).fillna(0).astype(int).T
df = df[sorted(df.columns, reverse = True)]
df['Total'] = df.sum(axis=1)
df.loc['Aantal per jaar'] = df.sum(axis=0)

display(df)


,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,Total
Fryslân,627,641,635,662,600,738,307,163,142,142,14,4671
Zuid-Holland,1719,2214,2431,2164,1763,2132,1352,1618,1051,220,0,16664
Zeeland,429,421,368,448,418,454,457,567,481,114,0,4157
Drenthe,361,523,296,102,145,153,124,149,95,58,0,2006
Noord-Holland,1574,1995,1685,698,175,177,107,46,59,45,0,6561
Flevoland,196,271,155,60,67,72,59,80,41,27,0,1028
Utrecht,555,692,763,805,814,869,831,1101,1002,576,0,8008
Groningen,506,708,687,166,113,126,106,116,97,28,0,2653
Gelderland,1440,2192,2083,2242,1763,2229,2059,2663,5174,2591,0,24436
Limburg,1040,1382,1277,1148,1189,1369,139,149,129,0,0,7822


Allereerst de meest standaard analyse. Het aantal beschikkingen per provincie per jaar. Hier valt dus op dat de ene organisatie deze beter heeft gepubliceerd dan de ander over de jaren heen. Gelderland heeft bijvoorbeeld de beschikkingen massaal gepubliceerd sinds 2014. Terwijl in Noord-Holland de aantallen pas rond 2020 echt zijn gaan stijgen (net als bij Noord-Brabant eigenlijk). Ook interessant is het totaal aantal per jaar waarbij wel opvalt dat dit wat wel te verwachten is steeds meer wordt (behalve die rare piek van Gelderland in 2015). 

In [42]:
doc_counts = {}

for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    for i in v:
        if org_name not in doc_counts:
            doc_counts[org_name]=0
        doc_counts[org_name] += len(i['infobox']['foi_files']['foi_documenten'])



df3 = pd.DataFrame({'Count': list(doc_counts.values())}, index=list(doc_counts.keys()))

df3['totaal'] = df['Total']
df3.loc['Alle Orgs'] = df3.sum(axis=0)
df3['fractie'] = round(df3['Count'] / df3['totaal'],2)


display(df3)
    

,Count,totaal,fractie
Fryslân,5038,4671,1.08
Zuid-Holland,18391,16664,1.10
Zeeland,4286,4157,1.03
Drenthe,2046,2006,1.02
Noord-Holland,8063,6561,1.23
Flevoland,1284,1028,1.25
Utrecht,11778,8008,1.47
Groningen,4232,2653,1.60
Gelderland,52664,24436,2.16
Limburg,14417,7822,1.84


Hoeveel documenten vinden we er per beschikking en zitten hier verschillen in per organisatie? Het valt hier op dat bij een aantal provincies eigenlijk bijna helemaal geen bijlagen te vinden zijn, maar bij een aantal juist een stuk vaker. Eigenlijk precies wat ik zou verwachten. Voor de duidelijkheid zou een fractie van 1 betekenen dat er helemaal geen bijlagen bestaan en een fractie van 2 betekent dat er gemiddeld 1 bijlage is per beschikking. 

In [48]:
doc_counts = {}

for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    for i in v:
        year = i['infobox']['dc_date_year']
        if year not in doc_counts:
            doc_counts[year]=0
        doc_counts[year] += len(i['infobox']['foi_files']['foi_documenten'])



df3 = pd.DataFrame({'Count': list(doc_counts.values())}, index=list(doc_counts.keys()))
df3['totaal p/j'] = df.loc['Aantal per jaar']
df3['Fractie'] = round(df3['Count'] / df3['totaal p/j'],2)


display(df3)
    

,Count,totaal p/j,Fractie
2013,15,14,1.07
2014,6957,3801,1.83
2015,14410,8338,1.73
2016,11527,6776,1.70
2017,9043,5856,1.54
2018,14322,9340,1.53
2019,11672,8141,1.43
2020,15373,9697,1.59
2021,18885,12449,1.52
2022,21235,14679,1.45


Ik wilde graag weten of bijlagen in de loop der jaren vaker of juist minder vaak voorkomen. Uit dit dataframe lijkt het dat er steeds minder bijlagen zijn gekomen per beschikking. Al is dit wel een beetje beinvloed vanwege het feit dat Gelderland de provincie is met het hoogste aantal bijlagen per beschikking en in 2014 en 2015 was veruit het grootste deel van alle beschikkingen van deze provincie. En dan is het dus logisch dat het aantal bijlagen ook hoger uitvalt. Het valt echter wel op dat de laatste 4 jaar het aantal bijlagen wel achteruit gaat. Dit zou wellicht kunnen komen door de terinlageleggingen. Als er meer van dit oort terinzageleggingen komen betekent dit dat er minder bijlagen voor altijd openbaar worden gemaakt. Omdat deze na verloop van de reactietermijn niet meer openbaar zijn. Voor hergebruik is dit dus een slechte zaak.

Het valt wel op dat de provincie Gelderland nu al in twee zaken het beste naar voren lijkt te komen.

In [6]:
import time
coord_counts = {}
for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    if org_name not in coord_counts:
        coord_counts[org_name]={'Point': 0, 'Line': 0, 'Polygon': 0}

    for i in v:
        geos = i['infobox']['foi_subjectGeograpgy']
        if 'POINT' in ' '.join(geos):
            coord_counts[org_name]['Point'] += 1
        if 'LINESTRING' in ' '.join(geos):
            coord_counts[org_name]['Line'] += 1
        if 'POLYGON' in ' '.join(geos):
            coord_counts[org_name]['Polygon'] += 1
            
    

df_c = pd.DataFrame(coord_counts).fillna(0).astype(int).T
df_c = df_c[sorted(df_c.columns, reverse = False)]
df_c.loc['Totaal aantal per type'] = df_c.sum(axis=0)

display(df_c)


,Line,Point,Polygon
Fryslân,183,990,117
Zuid-Holland,741,3833,332
Zeeland,78,709,10
Drenthe,290,406,149
Noord-Holland,452,3576,221
Flevoland,94,211,72
Utrecht,295,1065,103
Groningen,241,805,211
Gelderland,741,3348,391
Limburg,152,2260,461


**Titel: Het aantal keer dan een linestring, point en polygon voorkomen in de coordinaten van een beschikking per provincie.**

Het is ook belangrijk dat de coordinaten worden geanalyseerd. Omdat deze in 3 vormen te vinden zijn wordt in eerste instantie gekeken hoe vaak de verschillende vormen voorkomen per provincie en in totaal. Dit zijn de linestring (een lijn van coordinaten vaak een straat), een point (een enkel punt) en een polygon (gebiedsmarkering). Zoals te verwachten komt een enkel punt veruit het meeste voor, daarna een lijn en daarna een polygon. Het valt wel op dat bij twee provincies het percentage point een stuk lager is dan bij alle andere (Drenthe en Flevoland). Hierbij moet vermeld worden dat bij een enkele beschikking ook meerdere van deze vormen tegelijk kunnen bestaan. En deze vormen kunnen ook meerdere keren per beschikking voorkomen, maar dan is dit slechts voor 1 gerekend. 

In [7]:
coord_counts2 = {}
for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    if org_name not in coord_counts2:
        coord_counts2[org_name] = 0

    for i in v:
        if i['infobox']['foi_subjectGeograpgy'] != '':
            coord_counts2[org_name] += 1

df_coor = pd.DataFrame({'Count': list(coord_counts2.values())}, index=list(coord_counts2.keys()))
df_coor['Totaal'] = df['Total']
df_coor['fractie'] = round(df_coor['Count'] / df_coor['Totaal'],2)

display(df_coor)

,Count,Totaal,fractie
Fryslân,1600,4671,0.34
Zuid-Holland,5219,16664,0.31
Zeeland,1051,4157,0.25
Drenthe,1121,2006,0.56
Noord-Holland,4603,6561,0.70
Flevoland,584,1028,0.57
Utrecht,1667,8008,0.21
Groningen,1575,2653,0.59
Gelderland,4777,24436,0.20
Limburg,3098,7822,0.40


In dit dataframe is voor elke organisatie per beschikking gekeken of er bestaande coordinaten waren. En is daar het percentage waarbij dit wel zo was berekent. 

Het valt dus op dat dit nooit 100% is. Het valt voornamelijk op dat bij de organisaties die pas recent veel beschikkingen hebben gepubliceerd dit getal een stuk hoger is. Vermoedelijk zijn de coordinaten dus pas in een later stadium toegevoegd!

In [50]:
coord_counts3 = {}
for k,v in beschikkingen.items():
    for i in v:
        dc_date_year = i['infobox']['dc_date_year']
        if dc_date_year not in coord_counts3:
            coord_counts3[dc_date_year] = 0
            
        if i['infobox']['foi_subjectGeograpgy'] != '':
            coord_counts3[dc_date_year] += 1


df_coor2 = pd.DataFrame({'Count': list(coord_counts3.values())}, index=list(coord_counts3.keys()))
df_coor2['totaal p/j'] = df.loc['Aantal per jaar']

display(df_coor2)

https://zoek.officielebekendmakingen.nl/prb-2023-4


,Count,totaal p/j
2013,0,14
2014,0,3801
2015,0,8338
2016,0,6776
2017,0,5856
2018,0,9340
2019,0,8141
2020,0,9697
2021,7418,12449
2022,14679,14679


En dat blijkt dus ook uit dit dataframe. Hierin is het aantal beschikkingen per jaar met coordinaten te vinden. Het blijkt dus dat in 2022 en 2023 quasi elke beschikking coordinaten had. Het is dus zeer aannemelijk dat deze functie ergens in 2021 is geintroduceerd en dat sindsdien vrijwel alle beschikkingen coordinaten hadden. 

p.s. Ik was benieuwd waarom er slechts 1 beschikking in 2023 was die geen coordinaten had dat is deze: [beschikking](https://zoek.officielebekendmakingen.nl/prb-2023-4). Het blijkt dat hier eigenlijk wel coordinaten beschikbaar zijn, maar dat er een unieke metadata is gevonden: <metadata name="DC.spatial" scheme="OVERHEID.EPSG28992" content="150977.395 412545.95". Dit is dus een heel ander systeem van coordinaten dan het POINT() systeem wat alle andere items gebruiken. Ik zal nog onderzoeken of dit systeem voor 2021 wel van toepassing was! **UPDATE dit lijkt daadwerkelijk zo te zijn. (wat eigenlijk vraagt om een update van de scraper en een rerun van 2013 - 2021)**

In [36]:
first_day = ''
for k,v in beschikkingen.items():
    for i in v:
        if i['infobox']['foi_subjectGeograpgy'] != '':
            DATE = i['infobox']['foi_publishedDate']
            DATE = datetime.strptime(DATE, "%Y-%m-%d")
            if first_day == '':
                first_day = DATE
            else:
                if DATE < first_day:
                    first_day = DATE
            break

print(first_day)

2021-07-01 00:00:00


De allereerste versie van de nieuwe coordinaten is te vinden op 1 juli 2021. Hierom zal ik om mijn hypothese, dat hierna vrijwel alles onder het nieuwe systeem valt, proberen te bevestigen door de jaren analyse nogmaals te doen maar nu vanaf juli 2021.

In [39]:
from datetime import datetime

coord_counts4 = {}
year_counts = {}
for k,v in beschikkingen.items():
    for i in v:
        if datetime.strptime(i['infobox']['foi_publishedDate'], "%Y-%m-%d") >= datetime(2021, 7, 1):
            dc_date_year = i['infobox']['dc_date_year']
            if dc_date_year not in year_counts:
                year_counts[dc_date_year] = 0
                
            if dc_date_year not in coord_counts4:
                coord_counts4[dc_date_year] = 0
                
            if i['infobox']['foi_subjectGeograpgy'] != '':
                coord_counts4[dc_date_year] += 1

            year_counts[dc_date_year] += 1
            

df_coor3 = pd.DataFrame({'Count': list(coord_counts4.values())}, index=list(coord_counts4.keys()))
df_coor3['Total Per Year'] = year_counts
df_coor3['fractie'] = round(df_coor3['Count'] / df_coor3['Total Per Year'],2)

display(df_coor3)

,Count,Total Per Year,fractie
2021,7418,7546,0.98
2022,14679,14679,1.00
2023,11278,11279,1.00


En inderdaaad dit blijkt ongeveer zo te zijn. Ik vermoed dat het grootste deel van de missende items in 2021 uit juli 2021 zelf komen omdat de meeste provincies iets later zijn begonnen met het nieuwe coordinaten systeem. Dus sinds dien zijn eigenlijk alle coordinaten op deze manier verzameld. 

In [40]:
desc_counts = {}

for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    for i in v:
        if org_name not in desc_counts:
            desc_counts[org_name]=0

        if i['infobox']['dc_description'] != '':
            desc_counts[org_name]+=1


df_desc = pd.DataFrame({'Count': list(desc_counts.values())}, index=list(desc_counts.keys()))

df_desc['totaal'] = df['Total']

df_desc.loc['In totaal'] = df_desc.sum(axis=0)

df_desc['fractie'] = round(df_desc['Count'] / df_desc['totaal'],2)

display(df_desc)

,Count,totaal,fractie
Fryslân,10,4671,0.00
Zuid-Holland,6670,16664,0.40
Zeeland,617,4157,0.15
Drenthe,152,2006,0.08
Noord-Holland,2858,6561,0.44
Flevoland,14,1028,0.01
Utrecht,1674,8008,0.21
Groningen,0,2653,0.00
Gelderland,166,24436,0.01
Limburg,2456,7822,0.31


Omdat het downloaden van de files zelf erg moeilijk ging zijn is het eigenlijk niet mogelijk om een dc_description uit de tekst zelf te halen. Toch is in een deel van de beschikkingen deze description al aangeleverd. Dit is helaas maar in 22% van de gevallen zo bij de provincies. Waarbij een aantal prvincies dit redelijk vaak doen en een aantal eigenlijk nooit. Wederom valt op dat NH en NB dit verassend vaak doen. Zou dit een recentere ontwikkeling zijn? 

In [53]:
desc_counts = {}

for k,v in beschikkingen.items():
    k_split = k.split('.') 
    org_name = [key for key, val in organisaties.items() if val == k_split[1]][0]
    for i in v:
        year = i['infobox']['dc_date_year']
        if year not in desc_counts:
            desc_counts[year]=0

        if i['infobox']['dc_description'] != '':
            desc_counts[year]+=1


df_desc = pd.DataFrame({'Count': list(desc_counts.values())}, index=list(desc_counts.keys()))

df_desc['totaal'] = df.loc['Aantal per jaar']
df_desc['fractie'] = round(df_desc['Count'] / df_desc['totaal'],2)

display(df_desc)

,Count,totaal,fractie
2013,0,14,0.00
2014,10,3801,0.00
2015,164,8338,0.02
2016,299,6776,0.04
2017,330,5856,0.06
2018,1684,9340,0.18
2019,1617,8141,0.20
2020,1874,9697,0.19
2021,4117,12449,0.33
2022,5244,14679,0.36


Hieruit blijkt dus dat dit inderdaad steeds vaker voorkomt! (maar kan dus ook komen doordat vooral NB en NH dit doen en die zijn sterk in opkomst de laatste jaren)